In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import YouTubeVideo
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup


c:\Users\olive\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
def scrape(driver,id):
    '''
    Sets up a function for scraping data based on a given driver

    args: driver (defines which driver to scrape from)
    '''

    soup = BeautifulSoup(driver.page_source, 'lxml')
    table_node = soup.find(id=id)

    columns_html = table_node.thead.find_all('th')
        # Extract the text
    columns = []
    for col in columns_html:
        columns.append(col.text)
    columns = columns[7:]

    rows_list = table_node.tbody.find_all('tr')

    data = []
    fejl = []
    for row_node in rows_list:
        # if (rows_list.index(row_node) % 25 == 0):
        try:
            row = []
            for child in row_node.children:
                row.append(child.text)
            data.append(row)
        except:
            print(rows_list.index(row_node))
            fejl.append(rows_list.index(row_node)) # Column appear every 25th row
    
    return columns, data

def clean(columns, data,period):
    df_out = pd.DataFrame(data,columns=columns)\
        .iloc[:,:-1]\
        .assign(Nation= lambda x: x['Nation'].str.split(' ').str[1], 
            Pos = lambda x: pd.Categorical(x['Pos']),
            Squad = lambda x: pd.Categorical(x['Squad']),
            Year = period[0:4])
            # Nation = lambda x: pd.Categorical(x['Nation']))

    return df_out

In [3]:
def get_data(periods):
    df_list = []
    id = 'stats_standard'
    for period in periods:
        time.sleep(10)
        url = f'https://fbref.com/en/comps/9/{period}/stats/{period}-Premier-League-Stats'
        driver = webdriver.Chrome(ChromeDriverManager().install()) #driver
        driver.get(url)
        time.sleep(3)
        cookie = driver.find_element(By.CLASS_NAME,'qc-cmp2-summary-buttons').click()

        columns, data = scrape(driver,id)

        df_list.append(clean(columns, data, period))

    main_df = pd.concat(df_list)
    print(f'Number of observations in main dataset is: {len(main_df)}')
    return main_df


In [4]:
years = ['2018-2019','2019-2020','2020-2021','2021-2022']
main_df = get_data(years)

<ipython-input-3-6fea03c6f6aa>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install()) #driver


25
51
77
103
129
155
181
207
233
259
285
311
337
363
389
415
441
467
493
519
25
51
77
103
129
155
181
207
233
259
285
311
337
363
389
415
441
467
493
519
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
51
77
103
129
155
181
207
233
259
285
311
337
363
389
415
441
467
493
519
545
Number of observations in main dataset is: 2108


In [5]:
main_df.to_csv('main.csv')

In [ ]:
def scrape(driver):

    soup = BeautifulSoup(driver.page_source, 'lxml')
    table_node = soup.find(id='stats_standard')

    columns_html = table_node.thead.find_all('th')
        # Extract the text
    columns = []
    for col in columns_html:
        columns.append(col.text)
    columns = columns[7:]

    rows_list = table_node.tbody.find_all('tr')

    data = []
    fejl = []
    for row_node in rows_list:
        # if (rows_list.index(row_node) % 25 == 0):
        try:
            row = []
            for child in row_node.children:
                row.append(child.text)
            data.append(row)
        except:
            print(rows_list.index(row_node))
            fejl.append(rows_list.index(row_node)) # Column appear every 25th row
    
    return columns, data
        